<a href="https://www.kaggle.com/code/shedai/intro-textprocesing?scriptVersionId=246331513" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import re 
s = "Turkcell"

match = re.search("cell", s) 

print("Start Index:", match.start()) 
print("End Index:", match.end()) 

Start Index: 4
End Index: 8


In [3]:
import re 
s = "Turkcell@turkcell.com.tr"
match = re.search("^([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})", s) 
print("Start Index:", match.start()) 
print("End Index:", match.end()) 

Start Index: 0
End Index: 24


In [4]:
import re
regex = r"([a-zA-Z]+) (\d+)"

match = re.search(regex, "I was born on June 24")
if match != None:
    print ("Match at index %s, %s" % (match.start(), match.end()))
    print ("Full match: %s" % (match.group(0)))
    print ("Month: %s" % (match.group(1)))
    print ("Day: %s" % (match.group(2)))

else: 
    print ("The regex pattern does not match.")

Match at index 14, 21
Full match: June 24
Month: June
Day: 24


In [5]:
# creating the feature matrix 
from sklearn.feature_extraction.text import CountVectorizer
ans = "Turkcell ile bağlan hayata"
ans2 = "Turkcell ile dünyalar senin olsun"
matrix = CountVectorizer(max_features=1000)
X = matrix.fit_transform([ans]).toarray()
print(X)
X2 = matrix.transform([ans2]).toarray()
print(X2)

print(matrix.get_feature_names_out())

[[1 1 1 1]]
[[0 0 1 1]]
['bağlan' 'hayata' 'ile' 'turkcell']


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Örnek veri kümesi (pozitif ve negatif cümleler)
# Corpus / külliyat / derlem
texts = [
    "I love this product",       # pozitif
    "This is an amazing movie",  # pozitif
    "I am very happy with the service",  # pozitif
    "I hate this thing",         # negatif
    "This is the worst experience ever", # negatif
    "I am not happy with this",  # negatif
    "I feel wonderful", #positif
    "I love the product", #positif
]
labels = [1, 1, 1, 0, 0, 0, 1, 1]  # 1 = pozitif, 0 = negatif

# Metinleri vektöre çevir
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
# Veri setini eğitim/test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

# Lojistik regresyon modeli
model = LogisticRegression()
model.fit(X_train, y_train)

# Tahmin yap ve sonucu değerlendir
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Yeni metinleri tahmin et
new_texts = ["I love the experience", "I feel terrible", "What a wonderful day", "This is not good"]
new_X = vectorizer.transform(new_texts)
predictions = model.predict(new_X)

# Sonuçları göster
for text, label in zip(new_texts, predictions):
    sentiment = "Pozitif" if label == 1 else "Negatif"
    print(f"'{text}' -> {sentiment}")

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      0.50      0.50         2

    accuracy                           0.33         3
   macro avg       0.25      0.25      0.25         3
weighted avg       0.33      0.33      0.33         3

'I love the experience' -> Pozitif
'I feel terrible' -> Pozitif
'What a wonderful day' -> Pozitif
'This is not good' -> Negatif


In [7]:
from textblob import TextBlob

# İngilizce örnekler
texts = [
    "I love this product",
    "This is terrible",
    "What a great day!",
    "I hate this",
    "It's okay, not bad"
]

print("Sentiment Analizi (TextBlob):")
for text in texts:
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    label = "Pozitif" if polarity > 0 else "Negatif" if polarity < 0 else "Nötr"
    print(f"'{text}' -> {label} ({polarity:.2f})")

Sentiment Analizi (TextBlob):
'I love this product' -> Pozitif (0.50)
'This is terrible' -> Negatif (-1.00)
'What a great day!' -> Pozitif (1.00)
'I hate this' -> Negatif (-0.80)
'It's okay, not bad' -> Pozitif (0.42)
